In [ ]:
# final shape for test dataset (25788, 62)
# Takes about 35 seconds to run each file on my laptop

In [1]:
import xml.etree.ElementTree as ET
import xmltodict
import json
import time
import pandas as pd
import numpy as np

#pd.set_option('display.max_columns', 500)

import os
path = 'C:\Paresh\Data science\Lazaros\datasets'
os.chdir(path)

import glob
path = "C:\Paresh\Data science\Lazaros\datasets\*"

In [2]:
# helping functions which will be used later - only needs to be run once
# create a function to access dictionary values 2 levels deep

lst = [('Pose', '@Position', 'GazeData_Position'), 
       ('Pose', '@Rotation', 'GazeData_Rotation'),
       ('CombinedGazeRayWorld', '@Origin', 'GazeData_CombinedGazeRayWorld_Origin'),
       ('CombinedGazeRayWorld', '@Direction', 'GazeData_CombinedGazeRayWorld_Direction'),
       ('OriginalGaze', '@DeviceTimeStamp', 'OriginalGaze_DeviceTimeStamp'),
       ('OriginalGaze', '@SystemTimeStamp', 'OriginalGaze_SystemTimeStamp')]

def two_V(v1, v2, new_name):
    l = []

    for i in range(0, len(df)):
        l.append([tuple(np.float(i) for i in el.strip('()').split(',')) for el in d2[i][v1][v2].split('),(')])
    l = [l[i][0] for i in range(0,len(df))]
    df[new_name] = l
    df[[new_name + '_' + str(i) for i in range(1,len(l[0])+1)]] = pd.DataFrame(df[new_name].tolist())
    del df[new_name]
    
# create a function to access dictionary values 3 levels deep

lst2 = [('Left', 'GazeDirection', '@Value', 'GazeData_Left_GazeDirection'), 
        ('Left', 'GazeOrigin', '@Value', 'GazeData_Left_GazeOrigin'),
        ('Left', 'PupilDiameter', '@Value', 'GazeData_Left_PupilDiameter'), 
        ('Left', 'GazeRayWorld', '@Origin', 'GazeData_Left_GazeRayWorld_Origin'),
        ('Left', 'GazeRayWorld', '@Direction', 'GazeData_Left_GazeRayWorld_Direction'),
        ('Right', 'GazeDirection', '@Value', 'GazeData_Right_GazeDirection'), 
        ('Right', 'GazeOrigin', '@Value', 'GazeData_Right_GazeOrigin'),
        ('Right', 'PupilDiameter', '@Value', 'GazeData_Right_PupilDiameter'), 
        ('Right', 'GazeRayWorld', '@Origin', 'GazeData_Right_GazeRayWorld_Origin'),
        ('Right', 'GazeRayWorld', '@Direction', 'GazeData_Right_GazeRayWorld_Direction')]

def three_V(v1, v2, v3, new_name):
    l = []

    for i in range(0, len(df)):
        l.append([tuple(np.float(i) for i in el.strip('()').split(',')) for el in d2[i][v1][v2][v3].split('),(')])
    l = [l[i][0] for i in range(0,len(df))]
    df[new_name] = l
    df[[new_name + '_' + str(i) for i in range(1,len(l[0])+1)]] = pd.DataFrame(df[new_name].tolist())
    del df[new_name]
    
    
# create a function to access dictionary values 4 levels deep   

lst3 = [('OriginalGaze', 'LeftEye', 'GazeDirection', '@UnitVector', 'OriginalGaze_LeftEye_GazeDirection'),
        ('OriginalGaze', 'LeftEye', 'GazeOrigin', '@PositionInHMDCoordinates', 'OriginalGaze_LeftEye_GazeOrigin'),
        ('OriginalGaze', 'LeftEye', 'Pupil', '@PupilDiameter', 'OriginalGaze_LeftEye_PupilDiameter'),
        ('OriginalGaze', 'LeftEye', 'PupilPosition', '@PositionInTrackingArea', 'OriginalGaze_LeftEye_PupilPosition'),
        ('OriginalGaze', 'RightEye', 'GazeDirection', '@UnitVector', 'OriginalGaze_RightEye_GazeDirection'),
        ('OriginalGaze', 'RightEye', 'GazeOrigin', '@PositionInHMDCoordinates', 'OriginalGaze_RightEye_GazeOrigin'),
        ('OriginalGaze', 'RightEye', 'Pupil', '@PupilDiameter', 'OriginalGaze_RightEye_PupilDiameter'),
        ('OriginalGaze', 'RightEye', 'PupilPosition', '@PositionInTrackingArea', 'OriginalGaze_RightEye_PupilPosition')]


def four_V(v1, v2, v3, v4, new_name):
    l = []

    for i in range(0, len(df)):
        l.append([tuple(np.float(i) for i in el.strip('()').split(',')) for el in d2[i][v1][v2][v3][v4].split('),(')])
    l = [l[i][0] for i in range(0,len(df))]
    df[new_name] = l
    df[[new_name + '_' + str(i) for i in range(1,len(l[0])+1)]] = pd.DataFrame(df[new_name].tolist())
    del df[new_name]

In [3]:
# iterate for each file in desktop folder

count = 0
for filename in glob.glob(path):
    count += 1
    
    # convert to json
    tree = ET.parse(filename)
    xml_data = tree.getroot()
    xmlstr = ET.tostring(xml_data, encoding='utf8', method='xml')
    data = dict(xmltodict.parse(xmlstr))

    # extract the useful part of the data
    d = list(data.values())[0]
    d2 = list(d.values())[0]

    # create df
    
    # first create columns for timestamp, participant and day
    Timestamp = []
    for i in range(0, len(d2)):
        Timestamp.append(eval(d2[0]['@TimeStamp']))
        
    df = pd.DataFrame(Timestamp, columns = ['Time'])
    
    participant = filename[:-6] 
    day = filename[-5:-4]
    df['Participant'] = participant
    df['Day'] = day

    # reorder columns
    cols = list(df)
    cols.insert(0, cols.pop(cols.index('Participant')))
    cols.insert(1, cols.pop(cols.index('Day')))
    df = df.loc[:, cols]

    # add other columns
    for a, b, c in lst:
        two_V(a, b, c)

    for a, b, c, d in lst2:
        three_V(a, b, c, d)   

    for a, b, c, d, e in lst3:
        four_V(a, b, c, d, e)

    # export csv file
    df.to_csv('participant_' + str(count) + '.csv')
    print('Completed: Participant ' + str(count))

Completed: Participant 1
Completed: Participant 2
